In [52]:
from shapely.geometry import mapping
import pickle
import urllib

from cogeo_mosaic.mosaic import MosaicJSON
from cogeo_mosaic.backends import MosaicBackend

### Input Data

First we need to download the NAIP data index produced by Microsoft

In [ ]:
class DownloadProgressBar():
    """
    https://stackoverflow.com/questions/37748105/how-to-use-progressbar-module-with-urlretrieve
    """
    
    def __init__(self):
        self.pbar = None

    def __call__(self, block_num, block_size, total_size):
        if not self.pbar:
            self.pbar = progressbar.ProgressBar(max_value=total_size)
            self.pbar.start()
            
        downloaded = block_num * block_size
        if downloaded < total_size:
            self.pbar.update(downloaded)
        else:
            self.pbar.finish()

# Download index
url = "https://naipeuwest.blob.core.windows.net/naip-index/rtree/tiles.p"
destination_filename = "tiles.p"
urllib.request.urlretrieve(url, destination_filename, DownloadProgressBar())            

In [ ]:
# Load tile index
with open("tiles.p", "rb") as f:
    tile_index = pickle.load(f)

## Create metadata dict

In [101]:
# Create Metadata index
# v002/al/2015/al_100cm_2015/30086/m_3008601_ne_16_1_20150804.tifv
list_of_files = {
    f: {
        "path": f"https://naipeuwest.blob.core.windows.net/naip/{f}",
        "geom": geom,
        "year": f.split("/")[2],
        "state": f.split("/")[1],
        "capture_date": f.split("/")[5].split("_")[5].rstrip(".tif"),
        "quadrangle": f.split("/")[5].split("_")[1],
        "resolution": int(f.split("/")[3].split("_")[1].rstrip("cm"))
    }
    for _, (f, geom) in tile_index.items()
}

### Create Yearly MosaicJSON

In [82]:
# There are 3 different resolutions
zooms = {
    50: {"minzoom": 12, "maxzoom": 18},
    60: {"minzoom": 12, "maxzoom": 18},
    100: {"minzoom": 12, "maxzoom": 17}
}

def create_year_mosaic(year: int):
    year_files = dict(filter(lambda x: x[1]["year"] == str(year), list_of_files.items()))

    resolutions = {f["resolution"] for _, f in year_files.items()}

    for res in resolutions:
        year_files_res = dict(filter(lambda x: x[1]["resolution"] == res, year_files.items()))
        features = [
            {
                "geometry": mapping(f["geom"]),
                "properties": {
                    "path": f["path"],
                },
                "type": "Feature",
            }
            for key, f in year_files_res.items()
        ]

        mosaicjson = MosaicJSON.from_features(features, **zooms[res])
        with MosaicBackend(f"naip_{year}_{res}cm.json.gz", mosaic_def=mosaicjson) as mosaic:
            mosaic.write(overwrite=True)

for year in [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]:
    create_year_mosaic(year)

### Create `Latest` MosaicJSON

In [102]:
# For 50-60cm resolution

files_res = dict(filter(lambda x: x[1]["resolution"] in [50, 60], list_of_files.items()))
features = [
    {
        "geometry": mapping(f["geom"]),
        "properties": {
            "path": f["path"],
            "quadrangle": f["quadrangle"],
            "year": f["year"],
            "capture_date": f["capture_date"],
            "resolution": f["resolution"],
        },
        "type": "Feature",
    }
    for key, f in files_res.items()
]
features = sorted(features, key=lambda item: item["properties"]["capture_date"], reverse=True)

def quad_filter(tile, dataset, geoms):
    quad = []
    out_dataset = []
    for d in dataset:
        q = d["properties"]["quadrangle"]
        if q not in quad:
            out_dataset.append(d)
            quad.append(q)

    return out_dataset

mosaicjson = MosaicJSON.from_features(features, minzoom=12, maxzoom=18, asset_filter=quad_filter)
with MosaicBackend(f"naip_latest_60cm.json.gz", mosaic_def=mosaicjson) as mosaic:
    mosaic.write(overwrite=True)

In [111]:
# For 100cm resolution

files_res = dict(filter(lambda x: x[1]["resolution"] in [100], list_of_files.items()))
features = [
    {
        "geometry": mapping(f["geom"]),
        "properties": {
            "path": f["path"],
            "quadrangle": f["quadrangle"],
            "year": f["year"],
            "capture_date": f["capture_date"],
            "resolution": f["resolution"],
        },
        "type": "Feature",
    }
    for key, f in files_res.items()
]
features = sorted(features, key=lambda item: item["properties"]["capture_date"], reverse=True)

def quad_filter(tile, dataset, geoms):
    quad = []
    out_dataset = []
    for d in dataset:
        q = d["properties"]["quadrangle"]
        if q not in quad:
            out_dataset.append(d)
            quad.append(q)

    return out_dataset

mosaicjson = MosaicJSON.from_features(features, minzoom=12, maxzoom=17, asset_filter=quad_filter)
with MosaicBackend(f"naip_latest_100cm.json.gz", mosaic_def=mosaicjson) as mosaic:
    mosaic.write(overwrite=True)

### Create `Latest` MosaicJSON with mixed resolution

In [116]:
features = [
    {
        "geometry": mapping(f["geom"]),
        "properties": {
            "path": f["path"],
            "quadrangle": f["quadrangle"],
            "capture_date": f["capture_date"],
            "resolution": f["resolution"],
        },
        "type": "Feature",
    }
    for key, f in list_of_files.items()
]

# Sort by resolution and by dates
features = sorted(
    features, key=lambda item: (item["properties"]["capture_date"], -item["properties"]["resolution"]),
    reverse=True
)

def quad_filter(tile, dataset, geoms):
    quad = []
    out_dataset = []
    for d in dataset:
        q = d["properties"]["quadrangle"]
        if q not in quad:
            out_dataset.append(d)
            quad.append(q)

    return out_dataset

mosaicjson = MosaicJSON.from_features(features, minzoom=12, maxzoom=18, asset_filter=quad_filter)
with MosaicBackend(f"naip_latest.json.gz", mosaic_def=mosaicjson) as mosaic:
    mosaic.write(overwrite=True)